In [1]:
# import modules
import copy
import time
from sys import path
from collections import defaultdict

In [2]:
import pandas as pd
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split

In [3]:
path.append('../code/src')

In [4]:
import svd_constraint
from post_rec import PostRec
from inter_rec import InterRec
from evaluate import Evaluation as ev

In [5]:
# constants
# file path
input_path = "../data/"
output_path = '../result/'
rate_file = 'reduced_rating_data.csv'
attr_file = 'recipe_data.csv'

# user/item max
user_max = 1000
food_max = 5000

# const count
const_count = 3

In [6]:
# Dictionary keys
rs1 = 'w/o Constraint'
rs2 = 'Post-Single-'
rs3 = 'Inter-Single-'
val_r1 = 'RMSE'
val_r2 = 'nDCG'
val_t1 = 'train time(s)'
val_t2 = 'exec time(s)'

In [7]:
val_dict = {
    val_r1: 0,
    val_r2: 0,
    val_t1: 0,
    val_t2: 0
}

keys = [rs1]
keys = keys + [rs2 + str(i) for i in range (1, 4)]
keys = keys + [rs3 + str(i) for i in range (1, 4)]

result = {key: copy.deepcopy(val_dict) for key in keys}

In [ ]:
# load previous result
result_df = pd.read_csv(output_path + 'evaluation_single.csv', index_col = 0).transpose()
result.update(result_df.to_dict())
result

# Evaluate RS w/o Constraint
Check Suprise SVD performance with given rating data

In [8]:
# Source From: https://github.com/NicolasHug/Surprise/blob/master/examples/top_n_recommendations.py
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    ret = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        ret[int(uid)] = [int(i) for i, r in top_n[uid]]
    
    return ret

In [9]:
def run_SVD(k):    
    # get data
    reader = Reader(line_format='user item rating', sep=',', rating_scale=(0, 5))
    data = Dataset.load_from_file(input_path + rate_file, reader = reader)
    
    train_set, test_set = train_test_split(data, test_size=0.25)
    predict_set = train_set.build_anti_testset()
    
    # use SVD algorithm
    algo = SVD()
    
    # train
    start = time.time()
    algo.fit(train_set)
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = algo.test(test_set)
    r1 = ev.calculate_rmse(predict_test)

    # get top-n for anti-test-set
    start = time.time()
    predict_full = algo.test(predict_set)
    top_n = get_top_n(predict_full, k)
    t2 = time.time() - start
    
    # predict nDCG for test-set
    rel_dict = defaultdict(list)
    for (u, i, r) in test_set:
        if r >= 4:
            rel_dict[int(u)].append(int(i))
    
    top_n_df = pd.DataFrame.from_dict(top_n, orient='index')
    top_n_df = top_n_df.reindex(columns=[x for x in range(0, k)])
    r2 = ev.calculate_ndcg(rel_dict, top_n_df, k)
    
    return r1, r2, t1, t2

In [10]:
r1, r2, t1, t2 = run_SVD(500)
result[rs1][val_r1] = r1
result[rs1][val_r2] = r2
result[rs1][val_t1] = t1
result[rs1][val_t2] = t2

# Evaluate Post-Rec w/ Single Type Constraint

In [14]:
def run_post(ctype, idx, k, save_result = True):    
    # PostRec applies constraint after the rating of each item is predicted
    rec = PostRec(input_path + rate_file, input_path + attr_file, 
                           input_path + 'const_' + str(ctype) + '.' + str(idx) + '.csv',
                            need_test = True)
    
    rec.get_data()  # get rating, attribute, recipe data
    rec.set_result_n(k)
    
    # train with data
    start = time.time()
    rec.train()  
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = rec.test_rmse()
    r1 = ev.calculate_rmse(predict_test)
       
    # get top-n for anti-test-set
    start = time.time()
    rec.test()
    top_n_df = rec.get_top_n()
    t2 = time.time() - start
    
    # calculate ndcg
    r2 = ev.calculate_ndcg(rec.get_rel(), top_n_df, k)
    
    if save_result:       
        top_n_df.to_csv(output_path + 'PostRec_' + str(ctype) + '.' + str(idx) + '.csv')
        
    return r1, r2, t1, t2

In [15]:
for i in range (1, 4):
    r1_sum = 0
    r2_sum = 0
    t1_sum = 0
    t2_sum = 0
    for j in range (1, const_count + 1):
        r1, r2, t1, t2 = run_post(i, j, 500, False)
        r1_sum = r1_sum + r1
        r2_sum = r2_sum + r2
        t1_sum = t1_sum + t1
        t2_sum = t2_sum + t2
        print('Const_'+str(i)+'.'+str(j)+" done")
        
    result[rs2+str(i)][val_r1] = r1_sum/const_count
    result[rs2+str(i)][val_r2] = r2_sum/const_count
    result[rs2+str(i)][val_t1] = t1_sum/const_count
    result[rs2+str(i)][val_t2] = t2_sum/const_count

Const_1.1 done
Const_1.2 done
Const_1.3 done
Const_2.1 done
Const_2.2 done
Const_2.3 done
Const_3.1 done
Const_3.2 done
Const_3.3 done


# Evaluate Inter-Rec w/ Single Type Constraint

In [19]:
def run_inter(ctype, idx, k, save_result = True):    
    # PostRec applies constraint after the rating of each item is predicted
    rec = InterRec(input_path + rate_file, input_path + attr_file, 
                           input_path + 'const_' + str(ctype) + '.' + str(idx) + '.csv', 
                            svd_constraint.CnstSVD(), need_test = True)
    
    rec.get_data()  # get rating, attribute, recipe data
    rec.set_result_n(k)
    
    # train with data
    start = time.time()
    rec.train()  
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = rec.test_rmse()
    r1 = ev.calculate_rmse(predict_test)
    
    # get top-n for anti-test-set
    start = time.time()
    rec.test()
    top_n_df = rec.get_top_n()
    t2 = time.time() - start

    # calculate nDCG
    r2 = ev.calculate_ndcg(rec.get_rel(), top_n_df, k)
    
    if save_result:       
        top_n_df.to_csv(output_path + 'InterRec_' + str(ctype) + '.' + str(idx) + '.csv')
        
    return r1, r2, t1, t2

In [20]:
for i in range (1, 4):
    r1_sum = 0
    r2_sum = 0
    t1_sum = 0
    t2_sum = 0
    for j in range (1, const_count + 1):
        r1, r2, t1, t2 = run_inter(i, j, 500, False)
        r1_sum = r1_sum + r1
        r2_sum = r2_sum + r2
        t1_sum = t1_sum + t1
        t2_sum = t2_sum + t2
        print('Const_'+str(i)+'.'+str(j)+" done")
        
    result[rs3+str(i)][val_r1] = r1_sum/const_count
    result[rs3+str(i)][val_r2] = r2_sum/const_count
    result[rs3+str(i)][val_t1] = t1_sum/const_count
    result[rs3+str(i)][val_t2] = t2_sum/const_count

Const_1.1 done
Const_1.2 done
Const_1.3 done
Const_2.1 done
Const_2.2 done
Const_2.3 done
Const_3.1 done
Const_3.2 done
Const_3.3 done


# Result

In [25]:
result_df = pd.DataFrame.from_dict(result, orient='index', columns=['RMSE', 'nDCG', 'train time(s)', 'exec time(s)'])
result_df.to_csv(output_path + 'evaluation_single.csv')
result_df

,RMSE,nDCG,train time(s),exec time(s)
w/o Constraint,0.706073,0.046131,0.269782,14.747428
Post-Single-1,4.592492,0.214925,0.265947,250.630925
Post-Single-2,0.989168,0.055393,0.266286,69.362382
Post-Single-3,0.839229,0.065824,0.259639,176.790504
Inter-Single-1,3.897495,0.224253,258.345484,10.339448
Inter-Single-2,0.927025,0.061335,268.810266,16.657367
Inter-Single-3,0.827441,0.066673,381.436334,19.531564
